# Discrete event simulation using Agents.jl
Simon Frost (@sdwfrost), 2024-12-06

## Introduction

The discrete event simulation approach, implemented using [`Agents.jl`](https://github.com/JuliaDynamics/Agents.jl) taken here is:

- Stochastic
- Continuous in time (using `EventQueueABM`; there is also `StandardABM` for discrete-time simulation in Agents.jl)
- Discrete in state

## Libraries

In [ ]:
using Agents
using Random
using Distributions
using DrWatson: @dict
using Plots
using BenchmarkTools

## Transitions

First, we have to define our agent, which has a `status` (`:S`,`:I`, or `:R`). The standard SIR model is mass-action i.e. assumes that the population is well-mixed, and so we base our agent on `NoSpaceAgent` (which also has a member `id`.

In [ ]:
@agent struct Person(NoSpaceAgent)
    status::Symbol
end;

This is the transmission function; note that it operates on susceptibles making contact, rather than being focused on infected. This is an inefficient way of doing things, but shows the parallels between the different implementations. Note that the model properties, such as the contact rate `c` and the transmission probability `β`, are accessed via `.`.

In [ ]:
function transmit!(agent, model)
    # Choose random individual
    alter = random_agent(model)
    if alter.status == :I && (rand() ≤ model.β)
        # An infection occurs
        agent.status = :I
    end
end;

This is the recovery function.

In [ ]:
function recover!(agent, model)
    agent.status = :R
end;

By default, Agents.jl will schedule events based on an exponential distribution, parameterized by the propensity function. For added flexibility, we define our own propensity functions for transmission and recovery.

In [ ]:
function transmit_propensity(agent, model)
    if agent.status == :S
        return model.c
    else
        return 0.0
    end
end

In [ ]:
function recovery_propensity(agent, model)
    if agent.status == :I
        return model.γ
    else
        return 0.0
    end
end;

In [ ]:
transmit_event = AgentEvent(action! = transmit!, propensity = transmit_propensity)
recovery_event = AgentEvent(action! = recover!, propensity = recovery_propensity);

In [ ]:
events = (transmit_event, recovery_event);

We need some reporting functions.

In [ ]:
susceptible(x) = count(i == :S for i in x)
infected(x) = count(i == :I for i in x)
recovered(x) = count(i == :R for i in x);

This utility function sets up the model, by setting parameter fields and adding agents to the model. The constructor to `StandardABM` here takes the agent, followed by the `agent_step!` function, the model properties (passed as a `Dict`, and a random number generator. Other more complex models might also take a `model_step!` function.

In [ ]:
function init_model(β::Float64, c::Float64, γ::Float64, N::Int64, I0::Int64, rng::AbstractRNG=Random.GLOBAL_RNG)
    properties = @dict(β,c,γ)
    model = EventQueueABM(Person, events; properties, rng)
    for i in 1:N
        if i <= I0
            s = :I
        else
            s = :S
        end
        p = Person(;id=i,status=s)
        p = add_agent!(p,model)
    end
    return model
end;

## Time domain

In [ ]:
tf = 40.0;

## Parameter values

In [ ]:
β = 0.05
c = 10.0
γ = 0.25;

## Initial conditions

In [ ]:
N = 1000
I0 = 10;

## Random number seed

In [ ]:
seed = 1234
rng = Random.Xoshiro(seed);

## Running the model

In [ ]:
abm_model = init_model(β, c, γ, N, I0, rng);

In [ ]:
to_collect = [(:status, f) for f in (susceptible, infected, recovered)]
abm_data, _ = run!(abm_model, tf; adata = to_collect);

## Plotting

In [ ]:
plot(abm_data[:,1], abm_data[:,2], label="S", xlab="Time", ylabel="Number")
plot!(abm_data[:,1], abm_data[:,3], label="I")
plot!(abm_data[:,1], abm_data[:,4], label="R")

## Benchmarking

In [ ]:
@benchmark begin
abm_model = init_model(β, c, γ, N, I0, rng)
abm_data, _ = run!(abm_model, tf; adata = to_collect)
end